<a href="https://colab.research.google.com/github/Pontibesties/Proyecto-Big-Data/blob/main/Reg_proyecto_1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.mllib.util import MLUtils
from pyspark import SparkConf, SparkContext
from pyspark.ml.feature import VectorAssembler
from pyspark.mllib.regression import LabeledPoint
from pyspark.ml.regression import LinearRegression

In [ ]:
accidentes = pd.read_csv("https://www.dropbox.com/s/vkbkwblnb9y0xld/accidentes.csv?dl=1", sep=",", encoding='utf-8')
arrestos = pd.read_csv("https://www.dropbox.com/s/0m2z2av8merwk5b/arrestos.csv?dl=1", sep=",", encoding='utf-8')
aire = pd.read_csv("https://www.dropbox.com/s/9kx2xha806uhfzd/calidadAire.csv?dl=1", sep=",", encoding='utf-8')
educacion = pd.read_csv("https://www.dropbox.com/s/tfs594vh2wa7wtl/educacion.csv?dl=1", sep=",", encoding='utf-8')
pobreza = pd.read_csv("https://www.dropbox.com/s/p7dust9sj5nvlch/pobreza.csv?dl=1", sep=",", encoding='utf-8')

In [ ]:
pobreza.head(5)

,SPORDER,CIT,SEX,ESR,NP,TEN,Boro,CitizenStatus,EST_Childcare,EST_IncomeTax,Ethnicity,NYCgov_Income,NYCgov_Pov_Stat,NYCgov_REL,NYCgov_Threshold,Off_Pov_Stat,PreTaxIncome_PU,TotalWorkHrs_PU
0,1,1,2,1.0,1,2,2,1,0.0,33580.543,4,81417.961,2,0,16241.600,2,127650.22,2
1,1,1,2,1.0,2,1,3,1,0.0,95503.234,1,228417.950,2,0,22900.654,2,346479.19,2
2,2,1,1,6.0,2,1,3,1,0.0,95503.234,1,228417.950,2,1,22900.654,2,346479.19,2
3,1,4,1,1.0,2,2,2,2,0.0,27574.365,1,117058.230,2,0,22900.654,2,155003.84,1
4,2,4,2,1.0,2,2,2,2,0.0,27574.365,1,117058.230,2,1,22900.654,2,155003.84,1


In [ ]:
# Se filtran datos importantes
pobreza_fltrd = pobreza.filter(items=['SPORDER', 'CIT', 'SEX', 'ESR', 'Boro', 'CitizenStatus', 'EST_Childcare', 'EST_IncomeTax', 'Ethnicity', 'NYCgov_Income', 'NYCgov_Pov_Stat', 'NYCgov_Threshold', 'Off_Pov_Stat', 'PreTaxIncome_PU' ])

In [ ]:
# Se llenan de 0 los nan
pobreza_fltrd = pobreza_fltrd.fillna(0)

In [ ]:
pobreza_fltrd = pobreza_fltrd.astype('float64')

In [ ]:
from pyspark.sql import SparkSession
#Create PySpark SparkSession
spark = SparkSession.builder \
    .master("local[1]") \
    .appName("SparkByExamples.com") \
    .getOrCreate()
#Create PySpark DataFrame from Pandas
sparkDF=spark.createDataFrame(pobreza_fltrd) 
sparkDF.printSchema()
sparkDF.show()

root
 |-- SPORDER: double (nullable = true)
 |-- CIT: double (nullable = true)
 |-- SEX: double (nullable = true)
 |-- ESR: double (nullable = true)
 |-- Boro: double (nullable = true)
 |-- CitizenStatus: double (nullable = true)
 |-- EST_Childcare: double (nullable = true)
 |-- EST_IncomeTax: double (nullable = true)
 |-- Ethnicity: double (nullable = true)
 |-- NYCgov_Income: double (nullable = true)
 |-- NYCgov_Pov_Stat: double (nullable = true)
 |-- NYCgov_Threshold: double (nullable = true)
 |-- Off_Pov_Stat: double (nullable = true)
 |-- PreTaxIncome_PU: double (nullable = true)

+-------+---+---+---+----+-------------+-------------+-------------+---------+-------------+---------------+----------------+------------+---------------+
|SPORDER|CIT|SEX|ESR|Boro|CitizenStatus|EST_Childcare|EST_IncomeTax|Ethnicity|NYCgov_Income|NYCgov_Pov_Stat|NYCgov_Threshold|Off_Pov_Stat|PreTaxIncome_PU|
+-------+---+---+---+----+-------------+-------------+-------------+---------+-------------+-----

In [ ]:
vectorAssembler = VectorAssembler(inputCols = ['CIT', 'SEX', 'ESR', 'Boro', 'CitizenStatus', 'EST_Childcare', 'EST_IncomeTax', 'Ethnicity', 'PreTaxIncome_PU'], outputCol = 'features')
vtrain = vectorAssembler.transform(sparkDF)
vtrain = vtrain.select(['features', 'SPORDER', 'NYCgov_Income', 'NYCgov_Threshold']) # Off_Pov_Stat
vtrain.show(3)

+--------------------+-------+-------------+----------------+
|            features|SPORDER|NYCgov_Income|NYCgov_Threshold|
+--------------------+-------+-------------+----------------+
|[1.0,2.0,1.0,2.0,...|    1.0|    81417.961|         16241.6|
|[1.0,2.0,1.0,3.0,...|    1.0|    228417.95|       22900.654|
|[1.0,1.0,6.0,3.0,...|    2.0|    228417.95|       22900.654|
+--------------------+-------+-------------+----------------+
only showing top 3 rows



In [ ]:
splits = vtrain.randomSplit([0.7, 0.3])
train_df = splits[0]
test_df = splits[1]

In [ ]:
lr = LinearRegression(featuresCol = 'features', labelCol='NYCgov_Income', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(train_df)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

Coefficients: [514.4970807870751,0.0,-71.6118075712933,0.0,-2001.0182395574132,-1.4566762785002694,0.1464755178478641,2283.3212441379574,0.46375065784767383]
Intercept: 22205.972161058653


In [ ]:
lr_predictions = lr_model.transform(test_df)
lr_predictions.select("prediction","NYCgov_Income","NYCgov_Threshold").show(5)
from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="NYCgov_Income",metricName="r2")
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(lr_predictions))

+------------------+-------------+----------------+
|        prediction|NYCgov_Income|NYCgov_Threshold|
+------------------+-------------+----------------+
| 47214.70418603954|    83549.164|       50108.082|
|36764.050756402445|     34512.52|       29096.334|
| 44630.08505548914|    54920.121|       42861.836|
| 46212.63049714334|    46381.277|         39037.0|
|41905.081230690936|    50590.238|       30845.143|
+------------------+-------------+----------------+
only showing top 5 rows

R Squared (R2) on test data = 0.952062


In [ ]:
predictions = lr_model.transform(test_df)
predictions.select("prediction","NYCgov_Income","NYCgov_Threshold", 'SPORDER').show()

+------------------+-------------+----------------+-------+
|        prediction|NYCgov_Income|NYCgov_Threshold|SPORDER|
+------------------+-------------+----------------+-------+
| 47214.70418603954|    83549.164|       50108.082|    6.0|
|36764.050756402445|     34512.52|       29096.334|    3.0|
| 44630.08505548914|    54920.121|       42861.836|    3.0|
| 46212.63049714334|    46381.277|         39037.0|    4.0|
|41905.081230690936|    50590.238|       30845.143|    3.0|
|38367.929272398476|    28234.467|       24508.658|    2.0|
|  54074.5629334042|    53969.586|       42861.836|    5.0|
|25069.999700115826|     8982.877|       37461.793|    3.0|
|25069.999700115826|     8982.877|       37461.793|    5.0|
| 29979.58856689779|    20101.656|       37461.793|    5.0|
| 33697.01791070461|    23906.908|       30845.143|    2.0|
| 79550.05481195843|    84027.109|       56929.145|    7.0|
|58897.996098439806|    73327.047|       30845.143|    3.0|
| 67461.03885714443|    51472.172|      

In [ ]:
### Se compara la predicción con el threshold. Si la predicción es menor al threshold se tiene una situación de pobreza

pred_df = predictions.toPandas()

/databricks/spark/python/pyspark/sql/pandas/conversion.py:92: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Unable to convert the field features. If this column is not necessary, you may consider dropping it or converting to primitive type before the conversion.
Direct cause: Unsupported type in conversion to Arrow: VectorUDT
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warnings.warn(msg)


In [ ]:
pred_df.head(5)

,features,SPORDER,NYCgov_Income,NYCgov_Threshold,prediction
0,"[1.0, 1.0, 0.0, 1.0, 1.0, 0.0, -10347.301, 2.0...",6.0,83549.164,50108.082,47214.704186
1,"[1.0, 1.0, 0.0, 1.0, 1.0, 0.0, -9268.4092, 4.0...",3.0,34512.520,29096.334,36764.050756
2,"[1.0, 1.0, 0.0, 1.0, 1.0, 0.0, -8266.3994, 2.0...",3.0,54920.121,42861.836,44630.085055
3,"[1.0, 1.0, 0.0, 1.0, 1.0, 0.0, -6988.2583, 4.0...",4.0,46381.277,39037.000,46212.630497
4,"[1.0, 1.0, 0.0, 1.0, 1.0, 0.0, -5603.96, 4.0, ...",3.0,50590.238,30845.143,41905.081231


In [ ]:
pred_df[pred_df['NYCgov_Threshold'] > pred_df['prediction']]

,features,SPORDER,NYCgov_Income,NYCgov_Threshold,prediction
0,"[1.0, 1.0, 0.0, 1.0, 1.0, 0.0, -10347.301, 2.0...",6.0,83549.164,50108.082,47214.704186
7,"[1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 4457....",3.0,8982.877,37461.793,25069.999700
8,"[1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 4457....",5.0,8982.877,37461.793,25069.999700
9,"[1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 4.0, 273.5...",5.0,20101.656,37461.793,29979.588567
19,"[1.0, 1.0, 0.0, 1.0, 1.0, 48263.754, 23874.57,...",3.0,81763.750,42861.836,37878.790176
...,...,...,...,...,...
20319,"[5.0, 2.0, 6.0, 2.0, 3.0, 0.0, -500.0, 4.0, 30...",2.0,34858.828,42861.836,41641.458962
20347,"[5.0, 2.0, 6.0, 3.0, 3.0, 20931.227, -10102.20...",3.0,38940.109,39037.000,2547.289954
20348,"[5.0, 2.0, 6.0, 4.0, 3.0, 0.0, -10224.793, 3.0...",2.0,23018.719,35044.000,30745.379887
20352,"[5.0, 2.0, 6.0, 4.0, 3.0, 0.0, -5908.1743, 3.0...",1.0,19246.145,46545.484,29310.431393


In [ ]:
pred_df['SPORDER'].mean()

Out[88]: 2.306605162430072

In [ ]:
# SPORDER es el num de personas en la casa. A mayor num de personas mayor la pobreza teniendo en cuenta las demás variables

In [ ]:
#ACCIDENTES

In [ ]:
accidentes.head(5)

,CRASH_DATE,CRASH_TIME,STATE_REGISTRATION,VEHICLE_TYPE,VEHICLE_MAKE,VEHICLE_YEAR,VEHICLE_OCCUPANTS,DRIVER_SEX,DRIVER_LICENSE_STATUS,PUBLIC_PROPERTY_DAMAGE,PUBLIC_PROPERTY_DAMAGE_TYPE,CONTRIBUTING_FACTOR_1,CONTRIBUTING_FACTOR_2
0,09/23/2019,8:15,NY,Station Wagon/Sport Utility Vehicle,TOYT -CAR/SUV,2002.0,1.0,M,Licensed,N,Unspecified,Driver Inattention/Distraction,Unspecified
1,05/02/2016,17:35,NY,4 dr sedan,MERZ -CAR/SUV,2015.0,2.0,M,Licensed,N,Unspecified,Driver Inattention/Distraction,Unsafe Lane Changing
2,10/24/2019,13:15,NY,Bus,FRHT-TRUCK/BUS,2006.0,13.0,M,Licensed,N,Unspecified,Unspecified,Unspecified
3,08/18/2016,12:39,NY,Station Wagon/Sport Utility Vehicle,FORD -CAR/SUV,2005.0,2.0,F,Licensed,N,Unspecified,Driver Inattention/Distraction,Unspecified
4,07/22/2016,15:40,NY,Convertible,VOLK -CAR/SUV,2013.0,1.0,M,Licensed,N,Unspecified,Unspecified,Unspecified


In [ ]:
# Se filtran datos importantes
accidentes_fltrd = accidentes.filter(items=['VEHICLE_YEAR', 'VEHICLE_OCCUPANTS', 'DRIVER_SEX', 'PUBLIC_PROPERTY_DAMAGE', 'CONTRIBUTING_FACTOR_1', 'CONTRIBUTING_FACTOR_2' ])

In [ ]:
# Se llenan de 0 los nan
accidentes_fltrd = accidentes_fltrd.fillna(0)

In [ ]:
from pyspark.sql import SparkSession
#Create PySpark SparkSession
spark = SparkSession.builder \
    .master("local[1]") \
    .appName("SparkByExamples.com") \
    .getOrCreate()
#Create PySpark DataFrame from Pandas
sparkDF=spark.createDataFrame(accidentes_fltrd) 
sparkDF.printSchema()
sparkDF.show()

root
 |-- VEHICLE_YEAR: double (nullable = true)
 |-- VEHICLE_OCCUPANTS: double (nullable = true)
 |-- DRIVER_SEX: string (nullable = true)
 |-- PUBLIC_PROPERTY_DAMAGE: string (nullable = true)
 |-- CONTRIBUTING_FACTOR_1: string (nullable = true)
 |-- CONTRIBUTING_FACTOR_2: string (nullable = true)

+------------+-----------------+----------+----------------------+---------------------+---------------------+
|VEHICLE_YEAR|VEHICLE_OCCUPANTS|DRIVER_SEX|PUBLIC_PROPERTY_DAMAGE|CONTRIBUTING_FACTOR_1|CONTRIBUTING_FACTOR_2|
+------------+-----------------+----------+----------------------+---------------------+---------------------+
|      2002.0|              1.0|         M|                     N| Driver Inattentio...|          Unspecified|
|      2015.0|              2.0|         M|                     N| Driver Inattentio...| Unsafe Lane Changing|
|      2006.0|             13.0|         M|                     N|          Unspecified|          Unspecified|
|      2005.0|              2.0| 

In [ ]:
vectorAssembler = VectorAssembler(inputCols = ['VEHICLE_YEAR', 'VEHICLE_OCCUPANTS'], outputCol = 'features')
vtrain = vectorAssembler.transform(sparkDF)
vtrain = vtrain.select(['features', 'VEHICLE_YEAR', 'VEHICLE_OCCUPANTS', 'DRIVER_SEX']) # Off_Pov_Stat
vtrain.show(3)

+-------------+------------+-----------------+----------+
|     features|VEHICLE_YEAR|VEHICLE_OCCUPANTS|DRIVER_SEX|
+-------------+------------+-----------------+----------+
| [2002.0,1.0]|      2002.0|              1.0|         M|
| [2015.0,2.0]|      2015.0|              2.0|         M|
|[2006.0,13.0]|      2006.0|             13.0|         M|
+-------------+------------+-----------------+----------+
only showing top 3 rows



In [ ]:
splits = vtrain.randomSplit([0.7, 0.3])
train_df = splits[0]
test_df = splits[1]

In [ ]:
lr = LinearRegression(featuresCol = 'features', labelCol='VEHICLE_OCCUPANTS', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(train_df)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

Coefficients: [0.0,0.9500922581906605]
Intercept: 0.06853673255341661


In [ ]:
lr_predictions = lr_model.transform(test_df)
lr_predictions.select("prediction","VEHICLE_OCCUPANTS","VEHICLE_YEAR", 'DRIVER_SEX').show(5)
from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="VEHICLE_OCCUPANTS",metricName="r2")
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(lr_predictions))

+------------------+-----------------+------------+----------+
|        prediction|VEHICLE_OCCUPANTS|VEHICLE_YEAR|DRIVER_SEX|
+------------------+-----------------+------------+----------+
| 1.018628990744077|              1.0|      1963.0|         M|
|2.9188135071253978|              3.0|      1967.0|         M|
| 1.018628990744077|              1.0|      1968.0|         M|
| 1.018628990744077|              1.0|      1969.0|         M|
| 1.018628990744077|              1.0|      1970.0|         M|
+------------------+-----------------+------------+----------+
only showing top 5 rows

R Squared (R2) on test data = 0.997509


In [ ]:
predictions = lr_model.transform(test_df)
predictions.select("prediction","VEHICLE_OCCUPANTS","VEHICLE_YEAR", 'DRIVER_SEX').show()

+-------------------+-----------------+------------+----------+
|         prediction|VEHICLE_OCCUPANTS|VEHICLE_YEAR|DRIVER_SEX|
+-------------------+-----------------+------------+----------+
|  1.018628990744077|              1.0|      1963.0|         M|
| 2.9188135071253978|              3.0|      1967.0|         M|
|  1.018628990744077|              1.0|      1968.0|         M|
|  1.018628990744077|              1.0|      1969.0|         M|
|  1.018628990744077|              1.0|      1970.0|         M|
| 1.9687212489347374|              2.0|      1973.0|         M|
|0.06853673255341661|              0.0|      1974.0|         M|
|  1.018628990744077|              1.0|      1974.0|         M|
|  1.018628990744077|              1.0|      1976.0|         M|
|  1.018628990744077|              1.0|      1976.0|         M|
| 1.9687212489347374|              2.0|      1977.0|         M|
|  1.018628990744077|              1.0|      1978.0|         M|
| 1.9687212489347374|              2.0| 

In [ ]:
### Se compara la predicción con el threshold. Si la predicción es menor al threshold se tiene una situación de pobreza

pred_df = predictions.toPandas()

/databricks/spark/python/pyspark/sql/pandas/conversion.py:92: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Unable to convert the field features. If this column is not necessary, you may consider dropping it or converting to primitive type before the conversion.
Direct cause: Unsupported type in conversion to Arrow: VectorUDT
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warnings.warn(msg)


In [ ]:
pred_df.head(5)

,features,VEHICLE_YEAR,VEHICLE_OCCUPANTS,DRIVER_SEX,prediction
0,"[1963.0, 1.0]",1963.0,1.0,M,1.018629
1,"[1967.0, 3.0]",1967.0,3.0,M,2.918814
2,"[1968.0, 1.0]",1968.0,1.0,M,1.018629
3,"[1969.0, 1.0]",1969.0,1.0,M,1.018629
4,"[1970.0, 1.0]",1970.0,1.0,M,1.018629


In [ ]:
pred_df[pred_df['VEHICLE_OCCUPANTS'] > pred_df['prediction']]

,features,VEHICLE_YEAR,VEHICLE_OCCUPANTS,DRIVER_SEX,prediction
1,"[1967.0, 3.0]",1967.0,3.0,M,2.918814
5,"[1973.0, 2.0]",1973.0,2.0,M,1.968721
10,"[1977.0, 2.0]",1977.0,2.0,M,1.968721
12,"[1981.0, 2.0]",1981.0,2.0,M,1.968721
13,"[1981.0, 3.0]",1981.0,3.0,M,2.918814
...,...,...,...,...,...
382013,"[2026.0, 2.0]",2026.0,2.0,F,1.968721
382017,"[2028.0, 2.0]",2028.0,2.0,F,1.968721
382020,"[2086.0, 3.0]",2086.0,3.0,M,2.918814
382029,"[2718.0, 3.0]",2718.0,3.0,M,2.918814


In [ ]:
pred_df['VEHICLE_OCCUPANTS'].mean()

Out[20]: 1.3528287761950382

In [ ]:
# SPORDER es el num de personas en la casa. A mayor num de personas mayor la pobreza teniendo en cuenta las demás variables

In [ ]:
# EDUCACION

In [ ]:
educacion.head(5)

,Community School District,City Council District,School Name,# of students in grades 9-12,# of students in grades 9-12 scheduled for at least one semester of health instruction,%,# of 16-17 June and August graduates,# of 16-17 June and August graduates meeting high school health requirements,% 1
0,2,1,Emma Lazarus High School,306,218,71.2%,33,33,100.0%
1,2,1,Urban Assembly New York Harbor School,433,419,96.8%,70,70,100.0%
2,2,1,Essex Street Academy,328,239,72.9%,69,69,100.0%
3,1,1,Forsyth Satellite Academy,161,144,89.4%,s,s,s
4,2,1,Manhattan Early College School for Advertising,267,103,38.6%,s,s,s


In [ ]:
# Se filtran datos importantes
educacion_fltrd = educacion.filter(items=['Community School District', 'City Council District', '# of students in grades 9-12', '# of students in grades 9-12 scheduled for at least one semester of health instruction', '%', '# of 16-17 June and August graduates meeting high school health requirements', '% 1' ])

In [ ]:
# Se llenan de 0 los nan
educacion_fltrd = educacion_fltrd.fillna(0)

In [ ]:
from pyspark.sql import SparkSession
#Create PySpark SparkSession
spark = SparkSession.builder \
    .master("local[1]") \
    .appName("SparkByExamples.com") \
    .getOrCreate()
#Create PySpark DataFrame from Pandas
sparkDF=spark.createDataFrame(educacion_fltrd) 
sparkDF.printSchema()
sparkDF.show()

root
 |-- Community School District: long (nullable = true)
 |-- City Council District: long (nullable = true)
 |-- # of students in grades 9-12: long (nullable = true)
 |-- # of students in grades 9-12 scheduled for at least one semester of health instruction: string (nullable = true)
 |-- %: string (nullable = true)
 |-- # of 16-17 June and August graduates meeting high school health requirements: string (nullable = true)
 |-- % 1: string (nullable = true)

+-------------------------+---------------------+----------------------------+--------------------------------------------------------------------------------------+-----+----------------------------------------------------------------------------+------+
|Community School District|City Council District|# of students in grades 9-12|# of students in grades 9-12 scheduled for at least one semester of health instruction|    %|# of 16-17 June and August graduates meeting high school health requirements|   % 1|
+-----------------------

In [ ]:
vectorAssembler = VectorAssembler(inputCols = ['Community School District', 'City Council District', '# of students in grades 9-12' ], outputCol = 'features')
vtrain = vectorAssembler.transform(sparkDF)
vtrain = vtrain.select(['features', '# of students in grades 9-12', '# of 16-17 June and August graduates meeting high school health requirements', '% 1']) # Off_Pov_Stat
vtrain.show(3)

+---------------+----------------------------+----------------------------------------------------------------------------+------+
|       features|# of students in grades 9-12|# of 16-17 June and August graduates meeting high school health requirements|   % 1|
+---------------+----------------------------+----------------------------------------------------------------------------+------+
|[2.0,1.0,306.0]|                         306|                                                                          33|100.0%|
|[2.0,1.0,433.0]|                         433|                                                                          70|100.0%|
|[2.0,1.0,328.0]|                         328|                                                                          69|100.0%|
+---------------+----------------------------+----------------------------------------------------------------------------+------+
only showing top 3 rows



In [ ]:
splits = vtrain.randomSplit([0.7, 0.3])
train_df = splits[0]
test_df = splits[1]

In [ ]:
lr = LinearRegression(featuresCol = 'features', labelCol='# of students in grades 9-12', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(train_df)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

Coefficients: [0.0,0.0,0.9995775756366659]
Intercept: 0.23555970907214166


In [ ]:
lr_predictions = lr_model.transform(test_df)
lr_predictions.select("prediction", "# of students in grades 9-12").show(5)
from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="# of students in grades 9-12",metricName="r2")
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(lr_predictions))

+------------------+----------------------------+
|        prediction|# of students in grades 9-12|
+------------------+----------------------------+
|134.17895484438537|                         134|
|161.16754938657536|                         161|
|181.15910089930867|                         181|
| 212.1460057440453|                         212|
|261.12530695024196|                         261|
+------------------+----------------------------+
only showing top 5 rows

R Squared (R2) on test data = 1


In [ ]:
predictions = lr_model.transform(test_df)
predictions.select("prediction","# of students in grades 9-12").show()

+------------------+----------------------------+
|        prediction|# of students in grades 9-12|
+------------------+----------------------------+
|134.17895484438537|                         134|
|161.16754938657536|                         161|
|181.15910089930867|                         181|
| 212.1460057440453|                         212|
|261.12530695024196|                         261|
|267.12277240406195|                         267|
| 328.0970045178986|                         328|
| 360.0834869382719|                         360|
| 498.0251923761318|                         498|
| 653.9592941754516|                         654|
|3348.8204380919033|                        3350|
| 436.0513826866585|                         436|
| 368.0801075433652|                         368|
| 375.0771505728219|                         375|
|382.07419360227857|                         382|
| 416.0598311739252|                         416|
| 433.0526499597485|                         433|


In [ ]:
pred_df = predictions.toPandas()

In [ ]:
pred_df.head(5)

,features,# of students in grades 9-12,# of 16-17 June and August graduates meeting high school health requirements,% 1,prediction
0,"[1.0, 1.0, 134.0]",134,s,s,134.178955
1,"[1.0, 1.0, 161.0]",161,s,s,161.167549
2,"[1.0, 1.0, 181.0]",181,51,100.0%,181.159101
3,"[1.0, 1.0, 212.0]",212,52,100.0%,212.146006
4,"[2.0, 1.0, 261.0]",261,44,100.0%,261.125307


In [ ]:
pred_df[pred_df['# of students in grades 9-12'] > pred_df['prediction']]

,features,# of students in grades 9-12,# of 16-17 June and August graduates meeting high school health requirements,% 1,prediction
9,"[2.0, 1.0, 654.0]",654,142,100.0%,653.959294
10,"[2.0, 1.0, 3350.0]",3350,824,100.0%,3348.820438
18,"[10.0, 11.0, 1262.0]",1262,220,100.0%,1261.702460
27,"[9.0, 15.0, 581.0]",581,87,100.0%,580.990131
46,"[2.0, 2.0, 1727.0]",1727,358,99.7%,1726.506033
54,"[25.0, 20.0, 1706.0]",1706,283,100.0%,1705.514904
55,"[26.0, 20.0, 4414.0]",4414,924,100.0%,4412.370979
60,"[24.0, 26.0, 596.0]",596,143,100.0%,595.983795
61,"[25.0, 24.0, 723.0]",723,137,100.0%,722.930147
62,"[25.0, 24.0, 3493.0]",3493,674,100.0%,3491.760031


In [ ]:
pred_df['# of students in grades 9-12'].mean()

Out[29]: 552.078947368421

In [ ]:
#ARRESTOS

In [ ]:
arrestos.head(5)

,ARREST_DATE,PD_CD,PD_DESC,KY_CD,OFNS_DESC,LAW_CAT_CD,ARREST_BORO,ARREST_PRECINCT,JURISDICTION_CODE,AGE_GROUP,PERP_SEX,PERP_RACE
0,01/03/2022,205.0,"TRESPASS 2, CRIMINAL",352.0,CRIMINAL TRESPASS,M,K,75,0,18-24,M,BLACK
1,01/05/2022,397.0,"ROBBERY,OPEN AREA UNCLASSIFIED",105.0,ROBBERY,F,K,75,0,25-44,M,BLACK
2,01/06/2022,153.0,RAPE 3,104.0,RAPE,F,K,77,0,25-44,M,WHITE
3,01/08/2022,NaN,(null),NaN,(null),9,Q,113,3,45-64,M,WHITE HISPANIC
4,01/12/2022,NaN,(null),NaN,(null),9,Q,113,3,25-44,M,WHITE HISPANIC


In [ ]:
# Se filtran datos importantes
arrestos_fltrd = arrestos.filter(items=['ARREST_DATE', 'PD_CD', 'PD_DESC', 'KY_CD', 'OFNS_DESC', 'ARREST_BORO', 'ARREST_PRECINCT', 'JURISDICTION_CODE', 'AGE_GROUP', 'PERP_SEX', 'PERP_RACE' ])

In [ ]:
# Se llenan de 0 los nan
arrestos_fltrd = arrestos_fltrd.fillna(0)

In [ ]:
from pyspark.sql import SparkSession
#Create PySpark SparkSession
spark = SparkSession.builder \
    .master("local[1]") \
    .appName("SparkByExamples.com") \
    .getOrCreate()
#Create PySpark DataFrame from Pandas
sparkDF=spark.createDataFrame(arrestos_fltrd) 
sparkDF.printSchema()
sparkDF.show()

root
 |-- ARREST_DATE: string (nullable = true)
 |-- PD_CD: double (nullable = true)
 |-- PD_DESC: string (nullable = true)
 |-- KY_CD: double (nullable = true)
 |-- OFNS_DESC: string (nullable = true)
 |-- ARREST_BORO: string (nullable = true)
 |-- ARREST_PRECINCT: long (nullable = true)
 |-- JURISDICTION_CODE: long (nullable = true)
 |-- AGE_GROUP: string (nullable = true)
 |-- PERP_SEX: string (nullable = true)
 |-- PERP_RACE: string (nullable = true)

+-----------+-----+--------------------+-----+--------------------+-----------+---------------+-----------------+---------+--------+--------------+
|ARREST_DATE|PD_CD|             PD_DESC|KY_CD|           OFNS_DESC|ARREST_BORO|ARREST_PRECINCT|JURISDICTION_CODE|AGE_GROUP|PERP_SEX|     PERP_RACE|
+-----------+-----+--------------------+-----+--------------------+-----------+---------------+-----------------+---------+--------+--------------+
| 01/03/2022|205.0|TRESPASS 2, CRIMINAL|352.0|   CRIMINAL TRESPASS|          K|             75| 

In [ ]:
vectorAssembler = VectorAssembler(inputCols = ['PD_CD', 'KY_CD', 'ARREST_PRECINCT', 'JURISDICTION_CODE' ], outputCol = 'features')
vtrain = vectorAssembler.transform(sparkDF)
vtrain = vtrain.select(['features', 'ARREST_DATE', 'PD_CD', 'PD_DESC', 'KY_CD', 'OFNS_DESC', 'ARREST_BORO', 'ARREST_PRECINCT', 'JURISDICTION_CODE', 'AGE_GROUP', 'PERP_SEX', 'PERP_RACE']) # Off_Pov_Stat
vtrain.show(3)

+--------------------+-----------+-----+--------------------+-----+-----------------+-----------+---------------+-----------------+---------+--------+---------+
|            features|ARREST_DATE|PD_CD|             PD_DESC|KY_CD|        OFNS_DESC|ARREST_BORO|ARREST_PRECINCT|JURISDICTION_CODE|AGE_GROUP|PERP_SEX|PERP_RACE|
+--------------------+-----------+-----+--------------------+-----+-----------------+-----------+---------------+-----------------+---------+--------+---------+
|[205.0,352.0,75.0...| 01/03/2022|205.0|TRESPASS 2, CRIMINAL|352.0|CRIMINAL TRESPASS|          K|             75|                0|    18-24|       M|    BLACK|
|[397.0,105.0,75.0...| 01/05/2022|397.0|ROBBERY,OPEN AREA...|105.0|          ROBBERY|          K|             75|                0|    25-44|       M|    BLACK|
|[153.0,104.0,77.0...| 01/06/2022|153.0|              RAPE 3|104.0|             RAPE|          K|             77|                0|    25-44|       M|    WHITE|
+--------------------+-----------+

In [ ]:
splits = vtrain.randomSplit([0.7, 0.3])
train_df = splits[0]
test_df = splits[1]

In [ ]:
lr = LinearRegression(featuresCol = 'features', labelCol='ARREST_PRECINCT', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(train_df)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

Coefficients: [0.0,0.0,0.9914448601600714,0.0]
Intercept: 0.53778651409489


In [ ]:
lr_predictions = lr_model.transform(test_df)
lr_predictions.select("prediction", 'PD_CD', 'KY_CD', 'ARREST_PRECINCT', 'JURISDICTION_CODE' ).show(5)
from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="ARREST_PRECINCT",metricName="r2")
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(lr_predictions))

+------------------+-----+-----+---------------+-----------------+
|        prediction|PD_CD|KY_CD|ARREST_PRECINCT|JURISDICTION_CODE|
+------------------+-----+-----+---------------+-----------------+
| 5.495010814895247|  0.0|  0.0|              5|                0|
|101.66516225042217|  0.0|  0.0|            102|                0|
|101.66516225042217|  0.0|  0.0|            102|                0|
|101.66516225042217|  0.0|  0.0|            102|                0|
|103.64805197074232|  0.0|  0.0|            104|                0|
+------------------+-----+-----+---------------+-----------------+
only showing top 5 rows

R Squared (R2) on test data = 0.999927


In [ ]:
predictions = lr_model.transform(test_df)
predictions.select("prediction","ARREST_PRECINCT").show()

+------------------+---------------+
|        prediction|ARREST_PRECINCT|
+------------------+---------------+
| 5.495010814895247|              5|
|101.66516225042217|            102|
|101.66516225042217|            102|
|101.66516225042217|            102|
|103.64805197074232|            104|
|106.62238655122253|            107|
|101.66516225042217|            102|
|112.57105571218295|            113|
|112.57105571218295|            113|
|112.57105571218295|            113|
|112.57105571218295|            113|
|112.57105571218295|            113|
|112.57105571218295|            113|
|112.57105571218295|            113|
|112.57105571218295|            113|
|112.57105571218295|            113|
| 47.13569494161825|             47|
| 34.24691175953732|             34|
| 44.16136036113803|             44|
| 83.81915476754088|             84|
+------------------+---------------+
only showing top 20 rows



In [ ]:
pred_df = predictions.toPandas()

/databricks/spark/python/pyspark/sql/pandas/conversion.py:92: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Unable to convert the field features. If this column is not necessary, you may consider dropping it or converting to primitive type before the conversion.
Direct cause: Unsupported type in conversion to Arrow: VectorUDT
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warnings.warn(msg)


In [ ]:
pred_df.head(5)

,features,ARREST_DATE,PD_CD,PD_DESC,KY_CD,OFNS_DESC,ARREST_BORO,ARREST_PRECINCT,JURISDICTION_CODE,AGE_GROUP,PERP_SEX,PERP_RACE,prediction
0,"(0.0, 0.0, 5.0, 0.0)",03/03/2022,0.0,(null),0.0,(null),M,5,0,25-44,F,BLACK HISPANIC,5.495011
1,"(0.0, 0.0, 102.0, 0.0)",01/07/2022,0.0,(null),0.0,(null),Q,102,0,18-24,M,BLACK HISPANIC,101.665162
2,"(0.0, 0.0, 102.0, 0.0)",02/15/2022,0.0,(null),0.0,(null),Q,102,0,25-44,M,BLACK,101.665162
3,"(0.0, 0.0, 102.0, 0.0)",02/21/2022,0.0,(null),0.0,(null),Q,102,0,25-44,M,BLACK,101.665162
4,"(0.0, 0.0, 104.0, 0.0)",01/23/2022,0.0,(null),0.0,(null),Q,104,0,45-64,M,WHITE,103.648052


In [ ]:
pred_df[pred_df['ARREST_PRECINCT'] > pred_df['prediction']]

,features,ARREST_DATE,PD_CD,PD_DESC,KY_CD,OFNS_DESC,ARREST_BORO,ARREST_PRECINCT,JURISDICTION_CODE,AGE_GROUP,PERP_SEX,PERP_RACE,prediction
1,"(0.0, 0.0, 102.0, 0.0)",01/07/2022,0.0,(null),0.0,(null),Q,102,0,18-24,M,BLACK HISPANIC,101.665162
2,"(0.0, 0.0, 102.0, 0.0)",02/15/2022,0.0,(null),0.0,(null),Q,102,0,25-44,M,BLACK,101.665162
3,"(0.0, 0.0, 102.0, 0.0)",02/21/2022,0.0,(null),0.0,(null),Q,102,0,25-44,M,BLACK,101.665162
4,"(0.0, 0.0, 104.0, 0.0)",01/23/2022,0.0,(null),0.0,(null),Q,104,0,45-64,M,WHITE,103.648052
5,"(0.0, 0.0, 107.0, 0.0)",02/03/2022,0.0,(null),0.0,(null),Q,107,0,25-44,M,WHITE,106.622387
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28001,"[969.0, 881.0, 104.0, 0.0]",06/02/2022,969.0,"TRAFFIC,UNCLASSIFIED INFRACTIO",881.0,OTHER TRAFFIC INFRACTION,Q,104,0,25-44,F,WHITE HISPANIC,103.648052
28002,"[969.0, 881.0, 111.0, 0.0]",05/09/2022,969.0,"TRAFFIC,UNCLASSIFIED INFRACTIO",881.0,OTHER TRAFFIC INFRACTION,Q,111,0,45-64,M,BLACK,110.588166
28003,"[969.0, 881.0, 112.0, 0.0]",06/28/2022,969.0,"TRAFFIC,UNCLASSIFIED INFRACTIO",881.0,OTHER TRAFFIC INFRACTION,Q,112,0,25-44,M,WHITE,111.579611
28004,"[969.0, 881.0, 121.0, 0.0]",06/18/2022,969.0,"TRAFFIC,UNCLASSIFIED INFRACTIO",881.0,OTHER TRAFFIC INFRACTION,S,121,0,25-44,F,BLACK HISPANIC,120.502615


In [ ]:
pred_df['ARREST_PRECINCT'].mean()

Out[44]: 63.05313147182746

In [ ]:
# AIRE

In [ ]:
aire.head(5)

,Indicator ID,Name,Measure,Measure Info,Geo Type Name,Geo Join ID,Geo Place Name,Time Period,Start_Date,Data Value
0,386,Ozone (O3),Mean,ppb,CD,313,Coney Island (CD13),Summer 2013,06/01/2013,34.64
1,386,Ozone (O3),Mean,ppb,CD,313,Coney Island (CD13),Summer 2014,06/01/2014,33.22
2,386,Ozone (O3),Mean,ppb,Borough,1,Bronx,Summer 2013,06/01/2013,31.25
3,386,Ozone (O3),Mean,ppb,Borough,1,Bronx,Summer 2014,06/01/2014,31.15
4,383,Sulfur Dioxide (SO2),Mean,ppb,CD,211,Morris Park and Bronxdale (CD11),Winter 2008-09,12/01/2008,5.89


In [ ]:
# Se filtran datos importantes
aire_fltrd = aire.filter(items=['Indicator ID', 'Name', 'Geo Join ID', 'Geo Place Name', 'Time Period', 'Data Value' ])

In [ ]:
# Se llenan de 0 los nan
aire_fltrd = aire_fltrd.fillna(0)

In [ ]:
from pyspark.sql import SparkSession
#Create PySpark SparkSession
spark = SparkSession.builder \
    .master("local[1]") \
    .appName("SparkByExamples.com") \
    .getOrCreate()
#Create PySpark DataFrame from Pandas
sparkDF=spark.createDataFrame(aire_fltrd) 
sparkDF.printSchema()
sparkDF.show()

root
 |-- Indicator ID: long (nullable = true)
 |-- Name: string (nullable = true)
 |-- Geo Join ID: long (nullable = true)
 |-- Geo Place Name: string (nullable = true)
 |-- Time Period: string (nullable = true)
 |-- Data Value: double (nullable = true)

+------------+--------------------+-----------+--------------------+--------------+----------+
|Indicator ID|                Name|Geo Join ID|      Geo Place Name|   Time Period|Data Value|
+------------+--------------------+-----------+--------------------+--------------+----------+
|         386|          Ozone (O3)|        313| Coney Island (CD13)|   Summer 2013|     34.64|
|         386|          Ozone (O3)|        313| Coney Island (CD13)|   Summer 2014|     33.22|
|         386|          Ozone (O3)|          1|               Bronx|   Summer 2013|     31.25|
|         386|          Ozone (O3)|          1|               Bronx|   Summer 2014|     31.15|
|         383|Sulfur Dioxide (SO2)|        211|Morris Park and B...|Winter 2008

In [ ]:
vectorAssembler = VectorAssembler(inputCols = ['Indicator ID', 'Geo Join ID', 'Data Value'], outputCol = 'features')
vtrain = vectorAssembler.transform(sparkDF)
vtrain = vtrain.select(['features', 'Indicator ID', 'Geo Join ID', 'Data Value']) # Off_Pov_Stat
vtrain.show(3)

+-------------------+------------+-----------+----------+
|           features|Indicator ID|Geo Join ID|Data Value|
+-------------------+------------+-----------+----------+
|[386.0,313.0,34.64]|         386|        313|     34.64|
|[386.0,313.0,33.22]|         386|        313|     33.22|
|  [386.0,1.0,31.25]|         386|          1|     31.25|
+-------------------+------------+-----------+----------+
only showing top 3 rows



In [ ]:
splits = vtrain.randomSplit([0.7, 0.3])
train_df = splits[0]
test_df = splits[1]

In [ ]:
lr = LinearRegression(featuresCol = 'features', labelCol='Data Value', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(train_df)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

Coefficients: [0.0,0.0,0.9857984274958567]
Intercept: 0.26970774337365505


In [ ]:
lr_predictions = lr_model.transform(test_df)
lr_predictions.select("prediction",'Indicator ID', 'Geo Join ID', "Data Value").show(5)
from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="Data Value",metricName="r2")
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(lr_predictions))

+------------------+------------+-----------+----------+
|        prediction|Indicator ID|Geo Join ID|Data Value|
+------------------+------------+-----------+----------+
|31.933553234540568|         375|        101|     32.12|
|34.881090532753184|         375|        101|     35.11|
|36.537231890946224|         375|        102|     36.79|
| 29.45919918152597|         375|        103|     29.61|
| 34.68393084725401|         375|        103|     34.91|
+------------------+------------+-----------+----------+
only showing top 5 rows

R Squared (R2) on test data = 0.999798


In [ ]:
predictions = lr_model.transform(test_df)
predictions.select("prediction","Data Value", 'Indicator ID', 'Geo Join ID',).show()

+------------------+----------+------------+-----------+
|        prediction|Data Value|Indicator ID|Geo Join ID|
+------------------+----------+------------+-----------+
|31.933553234540568|     32.12|         375|        101|
|34.881090532753184|     35.11|         375|        101|
|36.537231890946224|     36.79|         375|        102|
| 29.45919918152597|     29.61|         375|        103|
| 34.68393084725401|     34.91|         375|        103|
| 32.22929276278933|     32.42|         375|        104|
|40.569147459404284|     40.88|         375|        104|
| 37.64132612974158|     37.91|         375|        106|
|28.749424313728955|     28.89|         375|        107|
| 36.10348058284805|     36.35|         375|        107|
|  35.7584511332245|      36.0|         375|        108|
|28.818430203653666|     28.96|         375|        109|
|29.823944599699438|     29.98|         375|        112|
|27.418596436609548|     27.54|         375|        201|
|31.549091847817188|     31.73|

In [ ]:
pred_df = predictions.toPandas()

/databricks/spark/python/pyspark/sql/pandas/conversion.py:92: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Unable to convert the field features. If this column is not necessary, you may consider dropping it or converting to primitive type before the conversion.
Direct cause: Unsupported type in conversion to Arrow: VectorUDT
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warnings.warn(msg)


In [ ]:
pred_df.head(5)

,features,Indicator ID,Geo Join ID,Data Value,prediction
0,"[375.0, 101.0, 32.12]",375,101,32.12,31.933553
1,"[375.0, 101.0, 35.11]",375,101,35.11,34.881091
2,"[375.0, 102.0, 36.79]",375,102,36.79,36.537232
3,"[375.0, 103.0, 29.61]",375,103,29.61,29.459199
4,"[375.0, 103.0, 34.91]",375,103,34.91,34.683931


In [ ]:
pred_df[pred_df['Indicator ID'] > pred_df['prediction']]

,features,Indicator ID,Geo Join ID,Data Value,prediction
0,"[375.0, 101.0, 32.12]",375,101,32.12,31.933553
1,"[375.0, 101.0, 35.11]",375,101,35.11,34.881091
2,"[375.0, 102.0, 36.79]",375,102,36.79,36.537232
3,"[375.0, 103.0, 29.61]",375,103,29.61,29.459199
4,"[375.0, 103.0, 34.91]",375,103,34.91,34.683931
...,...,...,...,...,...
4906,"[652.0, 408.0, 5.1]",652,408,5.10,5.297280
4907,"[652.0, 410.0, 7.7]",652,410,7.70,7.860356
4908,"[661.0, 407.0, 2.8]",661,407,2.80,3.029943
4909,"[661.0, 409.0, 3.0]",661,409,3.00,3.227103


In [ ]:
pred_df['Data Value'].mean()

Out[65]: 19.44365165156709